In [2]:
from pyspark import SparkContext
from pyspark import SparkConf

conf = SparkConf().setAppName("apriori")
sc = SparkContext(conf=conf)

In [3]:
from itertools import chain, combinations


In [18]:
def prune(item, k, freqDict):
    t1 = set(item[0])
    t2 = set(item[1])
    t = t1.union(t2)
    if len(t) == k and tuple(t.difference(t1).union(t.difference(t2))) in freqDict:
        return t
    return None

In [29]:
def getCandSet(t1,freqDict,k):
    t2 = t1
    t = t1 | t2
    if len(t) == k and tuple(t.difference(t1).union(t.difference(t2))) in freqDict:
        return t
    return None



In [20]:
# generate association rules
# compute confidence score

def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

def getAssociationRule(item, suppDict,k):
    """
    item: tuple, k-item frequent 
    suppDict: freq set dict with support
    return rules: Association Rule
    """
    _item = set(item)
    rules = []
    _subsets =  [set(x) for x in subsets(_item) if len(x) < k]
    for i in _subsets:
        rightItem = i
        leftItem = _item.difference(rightItem)
        leftItem = leftItem if len(leftItem) > 1 else tuple(leftItem)[0]
        conf = suppDict[item]/suppDict[leftItem]
        r = repr(leftItem) + '|' + repr(rightItem)
        rules.append((r, conf))
    return rules
            

In [21]:
# rules = freqSet.flatMap(lambda x: getAssociationRule(x, suppDict, k))
# rules.take(5)

# # top 5 pairs
# newRules = rules.sortBy(lambda t: (-t[1],t[0]))
# newRules.take(n)

In [8]:
text = sc.textFile('browsing.txt')

In [9]:
text.take(5)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ',
 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ',
 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ']

In [10]:
minSupp = 100
freqSetDict = {}

# k = 1

In [11]:

freqSetCount1 = text.flatMap(lambda t: t.strip().split(' ')) \
    .map(lambda item: (item, 1)).reduceByKey(lambda x,y: x+y ) \
    .filter(lambda x: x[1] >= minSupp)

In [12]:
freqSet1 = freqSetCount1.map(lambda x: x[0])

In [13]:
freqSetDict.update(freqSetCount1.collect())

In [23]:
len(freqSet1.collect())

647

# K= 2

In [26]:
# transform data to [['','','']]
textlist = text.map(lambda x: x.strip().split(' '))
textlist.take(5)

[['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222'],
 ['GRO99222',
  'GRO12298',
  'FRO12685',
  'ELE91550',
  'SNA11465',
  'ELE26917',
  'ELE52966',
  'FRO90334',
  'SNA30755',
  'ELE17451',
  'FRO84225',
  'SNA80192'],
 ['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO86643'],
 ['ELE17451', 'ELE37798', 'FRO86643', 'GRO56989', 'ELE23393', 'SNA11465'],
 ['ELE17451',
  'SNA69641',
  'FRO86643',
  'FRO78087',
  'SNA11465',
  'GRO39357',
  'ELE28573',
  'ELE11375',
  'DAI54444']]

In [30]:
# freqSet2 = getFreqSet(freqSet1, textlist, freqSetDict, minSupp, 2)

In [31]:
# freqSet2.take(5)

[]

In [34]:
k = 2
candSet2 = freqSet1.cartesian(freqSet1).filter(lambda x: x[0] < x[1] )          
candSet2.take(5)

[('FRO11987', 'SNA90258'),
 ('FRO11987', 'SNA80192'),
 ('FRO11987', 'GRO73461'),
 ('FRO11987', 'FRO86643'),
 ('FRO11987', 'SNA69641')]

In [36]:
# count support
candSetCount2 = candSet2.cartesian(textlist) \
        .map(lambda item: (tuple(item[0]),1) if set(item[0]).issubset(set(item[1])) else None) \
        .filter(lambda x: x) \
        .reduceByKey(lambda x,y: x+y) 
candSetCount2.take(5)

[(('FRO11987', 'FRO61611'), 1),
 (('FRO11987', 'GRO87006'), 14),
 (('FRO11987', 'GRO65522'), 4),
 (('FRO11987', 'GRO82670'), 1),
 (('ELE52966', 'GRO73461'), 56)]

In [37]:
# get freq 
freqSetCount2 = candSetCount2.filter(lambda x: x[1] >= minSupp)
freqSetCount2.take(5)

# update freqSetDict
freqSetDict.update(freqSetCount2.collect())

# get freq set
freqSet2 = freqSetCount2.map(lambda x: x[0])
freqSet2.take(5)

[('GRO73461', 'SNA72163'),
 ('GRO73461', 'SNA80324'),
 ('GRO73461', 'SNA38068'),
 ('DAI22177', 'ELE66810'),
 ('DAI22177', 'DAI75645')]

In [40]:
# get AssociationRule
rules2 = freqSet2.flatMap(lambda x: getAssociationRule(x, freqSetDict, k))
rules2.take(5)

[("'GRO73461'|{'SNA72163'}", 0.07912270960577457),
 ("'SNA72163'|{'GRO73461'}", 0.26146788990825687),
 ("'GRO73461'|{'SNA80324'}", 0.15602443087173792),
 ("'SNA80324'|{'GRO73461'}", 0.18462549277266754),
 ("'GRO73461'|{'SNA38068'}", 0.05885619100499723)]

In [43]:
# top 5 pairs
newRules2 = rules2.sortBy(lambda t: (-t[1],t[0]))
newRules2.take(5)

[("'DAI93865'|{'FRO40251'}", 1.0),
 ("'GRO85051'|{'FRO40251'}", 0.999176276771005),
 ("'GRO38636'|{'FRO40251'}", 0.9906542056074766),
 ("'ELE12951'|{'FRO40251'}", 0.9905660377358491),
 ("'DAI88079'|{'FRO40251'}", 0.9867256637168141)]

# k=3

In [ ]:
k = 3
candSet3 = freqSet2.cartesian(freqSet2).filter(lambda x: x[0] < x[1] ) \
        .map(lambda item: prune(item)).filter(lambda x: x)     
candSet3.take(5)

In [44]:
# count support
candSetCount3 = candSet3.cartesian(textlist) \
        .map(lambda item: (tuple(item[0]),1) if set(item[0]).issubset(set(item[1])) else None) \
        .filter(lambda x: x) \
        .reduceByKey(lambda x,y: x+y) 
candSetCount3.take(5)

# get freq 
freqSetCount3 = candSetCount3.filter(lambda x: x[1] >= minSupp)
freqSetCount3.take(5)

# update freqSetDict
freqSetDict.update(freqSetCount3.collect())

# get freq set
freqSet3 = freqSetCount3.map(lambda x: x[0])
freqSet3.take(5)

# get AssociationRule
rules3 = freqSet.flatMap(lambda x: getAssociationRule(x, freqSetDict, k))
rules3.take(5)

# top 5 triples
newRules3 = rules.sortBy(lambda t: (-t[1],t[0]))
newRules3.take(5)

KeyboardInterrupt: 

In [ ]:
len(freqSet2.collect())